**Research Question 2**

How often is a Q1 fracture associated with an ECTP on the same layer?

## General Process
1. load files
2. Optional: Find pits with properties of interest
3. Create a list of dictionaries of properties of interest
4. Create a dataframe from the list of dictionaries

*Question: In the case of multiple CT and ECT results, which results should be used?*
* Do any tests


* Do any ECTs and CTs have P and Q1 on the same layer?
    If yes, then correlation


* Are ECTP and Q1 failure on the same layer?
* If multiple CTs, are they all the same shear quality?


*STEPS*
1. find pits with ECTP and CT with shear quality results
2. Find tests with failure on the same layer
3. If multiple of either test, select test with lowest fracture character

*FIRST QUESTION*
* Of pits that have ECTP results and CT results with Shear quality. When failure occurs on the same layer, how often (pie chart and/or percentage table) is the fracture character of the CT Q1?








In [1]:
# import libraries
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

# Add the directory containing snowpylot to your Python path
# Adjust the path to match your directory structure
from snowpylot import *


ModuleNotFoundError: No module named 'snowpylot'

In [10]:
print("Current working directory:", os.getcwd())
print("\nPython path:")
for path in sys.path:
    print(path)

Current working directory: c:\Users\Administrator\Desktop\SnowPilot Repo\SnowPilotAnalytics\demos

Python path:
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.752.0_x64__qbz5n2kfra8p0\python313.zip
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.752.0_x64__qbz5n2kfra8p0\DLLs
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.752.0_x64__qbz5n2kfra8p0\Lib
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.13_3.13.752.0_x64__qbz5n2kfra8p0
c:\Users\Administrator\Desktop\SnowPilot Repo\SnowPilotAnalytics\.venv

c:\Users\Administrator\Desktop\SnowPilot Repo\SnowPilotAnalytics\.venv\Lib\site-packages
c:\Users\Administrator\Desktop\SnowPilot Repo\SnowPilotAnalytics\.venv\Lib\site-packages\win32
c:\Users\Administrator\Desktop\SnowPilot Repo\SnowPilotAnalytics\.venv\Lib\site-packages\win32\lib
c:\Users\Administrator\Desktop\SnowPilot Repo\SnowPilotAnalytics\.venv\Lib\site-packages\Pythonwin
c:\Users\Administrator\Desk

In [20]:
# Load xml files

#folder_path = "snowpits_200_MT" # The latest 200 snow pits from MT
folder_path = "snowpits_1K" # latest 1K snowpits from SnowPilot

# Create a list of all CAAML files in the folder
caaml_files = [f for f in os.listdir(folder_path) if f.endswith('.xml')] # List of all CAAML files in the folder

In [ ]:
# Find pits with CT and ECT results

pits_with_CT_ECT = []

for file in caaml_files:
    file_path = folder_path + '/' + file
    pit = caaml_parser(file_path)

    if len(pit.stabilityTests.CT) > 0 and len(pit.stabilityTests.ECT) > 0:
        pits_with_CT_ECT.append(pit)


print(len(pits_with_CT_ECT)) ## How many pits have CT and ECT results?

In [ ]:
# ECT and CT results

results_list=[]
results_dict={}

for pit in pits_with_CT_ECT:

    ECTs = pit.stabilityTests.ECT
    CTs = pit.stabilityTests.CT

    for ect in ECTs:
        for ct in CTs: # Every combination of ECT and CT results
            if(ect.depthTop == ct.depthTop and ect.propogation == True): # If failure on the same level and ECT has propogation
                results_dict ={
                    'PitID': pit.pitID,
                    'ECT Score': ect.testScore,
                    'CT Shear Qual': ct.shearQuality
                }
                results_list.append(results_dict)
   
df = pd.DataFrame(results_list)
grouped=df.groupby('PitID')
print(grouped.sum())

df = df.drop_duplicates(subset=['PitID'], keep='first') # Drop duplicates per pit NOTE This keeps the first occurance, update?
print('Unique df')
print(df)


In [ ]:
import matplotlib.pyplot as plt

# Calculate value counts for ShearQual
shear_counts = df['CT Shear Qual'].value_counts()

# Create pie chart
plt.figure(figsize=(10, 8))
plt.pie(shear_counts.values, 
        labels=shear_counts.index,
        autopct='%1.1f%%',  # Show percentages with 1 decimal place
        startangle=90)      # Rotate start of pie to 90 degrees

# Add title
plt.title('Distribution of CT Shear Quality Results in Pits Where ECTP Occurs on the same Layer', pad=20)

# Equal aspect ratio ensures that pie is drawn as a circle
plt.axis('equal')

# Show the plot
plt.show()

# Optional: Print the actual counts and percentages
print("\nShear Quality Distribution:")
print(shear_counts)
print("\nPercentages:")
print((shear_counts/len(df['CT Shear Qual'])*100).round(1))